___
# Ciência dos Dados - PROJETO 2

___
## Rafael dos Santos

## José Antônio Bechara

## Arthur Alegro de Oliveira

___

## Problema:

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


## Importando bibliotecas:

In [1]:
from collections import Counter as frequence_counter
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import matplotlib.pyplot as plt
from pprint import pprint
import pandas as pd
import numpy as np
import os

## Filtrando banco de dados:

A filtragem do banco de dados foi feita da seguinte forma:

* Converter todas as letras em minúsculas (evita de termos contagem de duas palavras iguais em significado mas escritas diferentemente. Ex.: "casa" e "Casa")
* Manter somente letras e números (elimina quaisquer outros valores tais como caracteres especiais, pontuações, emojis, etc)


Note que a filtragem foi feita antes da separação entre base de teste e de treinamento por conveniência, uma vez que a filtragem teria que ser feita em ambas futuramente.

In [2]:
def database_cleaner(dataframe, column, export=False):
    
    # Alphanumerics symbols:
    alpha_numerics = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
                      'v', 'w', 'x', 'y', 'z', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', ' ']
    
    comments_list = []
    
    # Reading all comments in a dataframe's column:
    for comment in dataframe[column]:
        # Converting all letters to lowercase:
        comment = comment.lower()
        # Selecting only alphanumeric characters:
        char_list = [char for char in comment if char in alpha_numerics]
        # Generating clean comment:
        clean_comment = ''.join(char_list)
        
        #clean_comment_stem = [stemmer.stem(word) for word in clean_comment]
        
        #final_comment = '.'
        
        #comments_list.append(clean_comment_stemizados)
        comments_list.append(clean_comment)
        
    # Overwriting dataframe column with clean comments:
    dataframe[column] = comments_list
    
    # Exportation control:
    if export == False:
        return dataframe
    elif export == True:
        dataframe.to_excel('clean-spamham2019.xlsx')

In [3]:
database_cleaner(pd.read_excel('spamham2019.xlsx'), 'Email', export=True)
clean_df = pd.read_excel('clean-spamham2019.xlsx')
clean_df.head()

,Email,Class
0,go until jurong point crazy available only in ...,ham
1,ok lar joking wif u oni,ham
2,free entry in 2 a wkly comp to win fa cup fina...,spam
3,u dun say so early hor u c already then say,ham
4,nah i dont think he goes to usf he lives aroun...,ham


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando:     
* 75% dos dados para a parte Treinamento
* 25% dos dados para a parte Teste

In [4]:
def database_separator(database, test_size):
    # Database separation:
    training_data, testing_data = train_test_split(database, test_size=test_size)
    # Percentage variables:
    training_size = len(training_data)
    training_percent = round( (len(training_data)/len(database))*100, 3)
    testing_size = len(testing_data)
    testing_percent = round( (len(testing_data)/len(database))*100, 3)
    training_percent_spam = round(len(training_data.loc[training_data["Class"]=='spam'])/ len(clean_df)*100, 3)
    training_percent_spam_c = round(len(training_data.loc[training_data["Class"]=='ham'])/ len(clean_df)*100, 3)
    # Shows the separation's absolute and relative values:
    print("Training Size: {} ({} %)\nTest Size: {} ({} %)\nTotal Size: {} (100 %)\n".format(training_size, training_percent, testing_size, testing_percent, len(database)))
    print("Training Partials: {} % SPAM + {} % HAM".format(training_percent_spam, training_percent_spam_c))
    return training_data, testing_data

In [5]:
training_data, testing_data = database_separator(pd.read_excel('clean-spamham2019.xlsx'), test_size=0.25)

Training Size: 4179 (75.0 %)
Test Size: 1393 (25.0 %)
Total Size: 5572 (100 %)

Training Partials: 10.176 % SPAM + 64.824 % HAM


Utilizando somente as 

___
## 3. Classificador Naive-Bayes

In [35]:
spam_words = [word  for comment in (training_data[training_data['Class'] == 'spam']['Email']) for word in comment.split(' ') if word != ""]
ham_words = [word for comment in (training_data[training_data['Class'] == 'ham']['Email']) for word in comment.split(' ') if word != ""]
all_words = spam_words + spam_c_words

spam_freq = frequence_counter(spam_words)
ham_freq = frequence_counter(ham_words)

print("SPAM words: {}\nHAM words: {}".format(len(spam_words), len(ham_words)))

SPAM words: 13438
HAM words: 49982


Sabendo que todas as mensagens são necessariamente e somente SPAM ou HAM,  podemos dizer que suas probabilidades são complementares. Portanto:

$$ SPAM + HAM\space=\space Total$$

$$P(SPAM) + P(HAM)\space=\space 1$$

Assim a temos que:

$$P(SPAM)=\frac{SPAM}{Total}$$

$$P(HAM)=\frac{HAM}{Total}$$

In [ ]:
def P_SPAM(spam_words, all_words):
    return len(spam_words)/len(all_words)

def P_HAM(ham_words, all_words):
    return len(ham_words)/len(all_words)

Segundo o Teorema de Bayes, temos que:

$$P(A|B)\space=\space\frac{P(B|A)\cdot P(A)}{P(B)}$$

Portanto temos que:

$$P(SPAM|message)\space=\space\frac{P(message|SPAM)\cdot P(SPAM)}{P(message)}$$

$$P(HAM|message)\space=\space\frac{P(message|HAM)\cdot P(HAM)}{P(message)}$$

Considerando que o classificação consiste em saber qual probabilidade é maior (de ser SPAM ou HAM), basta dividir ambas as equações:

$$Class\space=\space\frac{P(SPAM|message)}{P(HAM|message)}\space=\space \frac{\frac{P(message|SPAM)\cdot P(SPAM)}{P(message)}}{\frac{P(message|HAM)\cdot P(HAM)}{P(message)}}\space=\space \frac{P(message|SPAM)\cdot P(SPAM)}{P(message|HAM)\cdot P(HAM)}$$

Com isso, eliminamos a variável $P(message)$ e, além disso, torna-se muito mais fácil a classificação, basta comparar se o resultado é maior ou menor que 0.

Note que se a probabilidade de ser SPAM é maior do que ser HAM, ou seja, se $P(message|SPAM) > P(message|HAM)$ o resultado será maior do que 0; caso contrário, ou seja, se $P(message|SPAM) < P(message|HAM)$ o resultado será menor do que 0.

In [ ]:
def classification(messagem, spam_words, ham_words, all_words):
    P_SPAM_message = P_message_SPAM()*P_SPAM(spam_words, all_words)
    P_HAM_message = P_message_HAM()*P_HAM(ham_words, all_words)
    if P_SPAM_message > P_HAM_message:
        return "SPAM"
    elif P_SPAM_message < P_HAM_message:
        return "HAM"
    else:
        return "-"

A contribuição de Naive vem a seguir: Assumir que todas as palavras de uma mesma mensagem sejam independentes entre si. Isso significa que a a mensagem nada mais é do que um conjunto de palavras (implicando também que a ordem dessas palavras é irrelevante). Em termos matemáticos podemos dizer que:

$$P(message)\space=\space\sum^{\infty}_{n=1} P(word_n)$$

Assim sendo podemos reescrever as equações obtidas anteriormente da seguinte forma:

$$P(message|SPAM)\space=\space\sum^{\infty}_{n=1} P(word_n|SPAM)$$

$$P(message|HAM)\space=\space\sum^{\infty}_{n=1} P(word_n|HAM)$$

In [ ]:
def P_word_SPAM(word):
    return spam_freq[word]/len(all_words)

def P_word_HAM(word):
    return ham_freq[word]/len(all_words)

message_words = [word for word in message.split('') if != ""]

In [ ]:
class naive_bayes_classifier():
    __init__():
         

___
## 4. Qualidade do Classificador alterando a base de treinamento